In [1]:
from scipy.sparse import *

import numpy as np

In [17]:
A = coo_matrix([[1,2,0],[0,0,3],[4,0,5]])
print(A)

  (0, 0)	1
  (0, 1)	2
  (1, 2)	3
  (2, 0)	4
  (2, 2)	5


In [16]:
B = A.tocsr()
type(B)

scipy.sparse._csr.csr_matrix

In [20]:
C = A.todense()
C

matrix([[1, 2, 0],
        [0, 0, 3],
        [4, 0, 5]])

In [21]:
v = np.array([1,0,-1])
A.dot(v)

array([ 1, -3, -1], dtype=int32)

In [23]:
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import solve, norm
from numpy.random import rand

In [24]:
A = lil_matrix((1000, 1000))
A[0, :100] = rand(100)
A[1, 100:200] = A[0, :100]
A.setdiag(rand(1000))

In [27]:
A.todense()

matrix([[0.12253826, 0.47962631, 0.11216353, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.35603151, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.27978352, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.81129307, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.42461823,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.92433658]])

In [28]:
A = A.tocsr()
b = rand(1000)
x = spsolve(A, b)

In [29]:
x

array([ 1.04195047e+03, -4.83917213e+02,  2.74851417e+00,  6.46275009e-01,
        3.87810311e-01,  8.45492300e-01,  1.39970489e+00,  9.97763245e-01,
        2.15093374e-01,  4.05547857e+01,  1.86127861e-01,  5.88417800e-01,
        2.12394741e+00,  8.32662626e-01,  2.59841091e+00,  3.73991658e-02,
        1.37473347e+00,  4.25435851e-01,  1.61628058e-02,  9.15466019e+00,
        3.55878041e-01,  3.68461263e+00,  2.78788666e+00,  1.14883954e+00,
        1.99390327e+00,  1.34398734e+00,  2.57932810e+00,  3.39745689e-02,
        8.00249835e-01,  1.48770763e+00,  1.68243842e-01,  1.51310269e+00,
        4.70736243e-01,  6.48042183e-01,  2.05242479e-01,  1.39852264e-01,
        3.52640731e+00,  8.35024653e-01,  1.18273292e+00,  8.68252639e-01,
        3.62916590e+00,  1.31894788e+00,  6.14109519e+00,  1.63112155e+00,
        2.83633456e-01,  6.52920530e-01,  1.95010438e-02,  1.14927155e+00,
        1.24764207e+00,  6.36272093e-01,  9.71988447e-01,  5.53552539e-01,
        6.63816303e-01,  

In [68]:
from scipy.optimize import minimize
import numpy as np
e = 1e-10 # 非常接近0的值
fun = lambda x : (x[0] - 0.667) / (x[0] + x[1] + x[2] - 2) # 约束函数
cons = ({'type': 'eq', 'fun': lambda x: x[0] * x[1] * x[2] - 1}, # xyz=1
        {'type': 'ineq', 'fun': lambda x: x[0] - 0}, # x>=e，即 x > 0
        {'type': 'ineq', 'fun': lambda x: x[1] - 0},
        {'type': 'ineq', 'fun': lambda x: x[2] - 0}
       )
x0 = np.array((1.0, 1.0, 1.0)) # 设置初始值
res = minimize(fun, x0, method='SLSQP', constraints=cons)
print('最小值：',res.fun)
print('最优解：',res.x)
print('迭代终止是否成功：', res.success)
print('迭代终止原因：', res.message)

最小值： -0.18814357989751096
最优解： [0.29250894 1.84897232 1.84897233]
迭代终止是否成功： True
迭代终止原因： Optimization terminated successfully


现有3支股票的收盘价P、收益率R和贝塔值数据b，资金一千万，在不允许卖空的情况下，希望投资组合收益率最大且组合贝塔<1.2，求每支股票的权重。

In [63]:
P=np.array([5.29,6.67,6.50])
R=np.array([0.105143,0.132796,0.055905])
b=np.array([1.31,1.241,1.06])
def f(w):#定义求最优值函数
	w=np.array(w)
	return -np.dot(R,w)#写np.sum(R*w)也可
cons=({'type':'eq','fun':lambda w:np.sum(w)-1},{'type':'ineq','fun':lambda w:1.2-np.dot(w,b)})
#等式约束：∑wi-1=0；不等式约束：1.2-∑wi*b≥0
bnds=((0,1),(0,1),(0,1))#每支股票权重的边界条件
result=minimize(f,[0.3,0.3,0.4],method='SLSQP',bounds=bnds,constraints=cons)
result

     fun: -0.11537870073023682
     jac: array([-0.105143, -0.132796, -0.055905])
 message: 'Optimization terminated successfully'
    nfev: 20
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.38777878e-16, 7.73480651e-01, 2.26519349e-01])

In [64]:
result.x.round(2)

array([0.  , 0.77, 0.23])

In [65]:
-result.fun

0.11537870073023682

In [66]:
stock=['股票a','股票b','股票c']
for i in range(3):
	print(stock[i],(10000000/P[i]*result.x[i]).round(0))


股票a 0.0
股票b 1159641.0
股票c 348491.0


In [69]:
import numpy as np
from scipy.optimize import minimize

# 定义目标函数
def func(x, sign=1.0):
    # scipy.minimize默认求最小，求max时只需要sign*(-1)，跟下面的args对应
    return sign * (x[0] ** 2 + x[1] ** 2 + x[2] ** 2)
    # return sign * (np.power(x[0], 2) + np.power(x[1], 2) + np.power(x[2], 2))

# 定义目标函数的梯度
def func_deriv(x, sign=1):
    jac_x0 = sign * (2 * x[0])
    jac_x1 = sign * (2 * x[1])
    jac_x2 = sign * (2 * x[2])
    return np.array([jac_x0, jac_x1, jac_x2])

# 定义约束条件
# constraints are defined as a sequence of dictionaries, with keys type, fun and jac.
cons = (
    {'type': 'eq',
     'fun': lambda x: np.array([x[0] + 2 * x[1] - x[2] - 4]),
     'jac': lambda x: np.array([1, 2, -1])},

    {'type': 'eq',
     'fun': lambda x: np.array([x[0] - x[1] - x[2] + 2]),
     'jac': lambda x: np.array([1, -1, -1])}
    )

In [71]:
# 定义初始解x0
x0 = np.array([-1.0, 1.0, 1.0])

# 使用SLSQP算法求解
res = minimize(func, x0 , args=(1,), jac=func_deriv, method='SLSQP', options={'disp': True},constraints=cons)
# args是传递给目标函数和偏导的参数，此例中为1，求min问题。args=-1时是求解max问题
print(res) 

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.000000000000002
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
     fun: 4.000000000000002
     jac: array([-4.44089210e-16,  4.00000000e+00, -1.33226763e-15])
 message: 'Optimization terminated successfully'
    nfev: 2
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([-2.22044605e-16,  2.00000000e+00, -6.66133815e-16])


In [72]:
>>> import numpy as np
>>> from scipy.optimize import minimize

In [75]:
>>> def rosen(x):
...     """The Rosenbrock function"""
...     return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
>>> x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
>>> res = minimize(rosen, x0, method='nelder-mead',
...                options={'xtol': 1e-8, 'disp': True})


Optimization terminated successfully.
         Current function value: 0.000066
         Iterations: 141
         Function evaluations: 243


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19024\2658159131.py:5: OptimizeWarning: Unknown solver options: xtol
  res = minimize(rosen, x0, method='nelder-mead',


In [77]:
print(res.x)

[0.99910115 0.99820923 0.99646346 0.99297555 0.98600385]
